In [85]:
import tensorflow as tf

import pandas as pd
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [86]:
data = pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [87]:
data.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [88]:
data.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [89]:
data.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [90]:
data.Survived.unique()

array([0, 1], dtype=int64)

In [91]:
data.Pclass.unique()

array([3, 1, 2], dtype=int64)

In [92]:
data.Sex.unique()

array(['male', 'female'], dtype=object)

In [93]:
data.SibSp.unique()

array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

In [94]:
data.Parch.unique()

array([0, 1, 2, 5, 3, 4, 6], dtype=int64)

In [96]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [97]:
data.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [98]:
cabin = data.pop('Cabin')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [99]:
data = data.dropna()
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [100]:
data.pop('Name')
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,female,35.0,1,0,113803,53.1000,S
4,5,0,3,male,35.0,0,0,373450,8.0500,S


In [101]:
data.pop('Ticket')
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [102]:
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

print(len(train))
print(len(val))
print(len(test))

455
114
143


In [103]:
def df_to_dataset(df, shuffle=False, batch_size=32):
    df = df.copy()
    labels = df.pop('Survived')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [104]:
train_ds = df_to_dataset(train, shuffle=True)
val_ds = df_to_dataset(val)
test_ds = df_to_dataset(test)

In [107]:
feature_columns = []

for header in ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']:
    feature_columns.append(feature_column.numeric_column(header))
    
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[10, 20, 30, 40, 50, 60, 70])
feature_columns.append(age_buckets)

sex = feature_column.categorical_column_with_vocabulary_list('Sex', ['male', 'female'])
sex_one_hot = feature_column.indicator_column(sex)
feature_columns.append(sex_one_hot)

embarked = feature_column.categorical_column_with_vocabulary_list('Embarked', ['S', 'C', 'Q'])
embarked_one_hot = feature_column.indicator_column(embarked)
feature_columns.append(embarked_one_hot)

In [108]:
feature_layer = layers.DenseFeatures(feature_columns)

In [110]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
15/15 [==============================] - 0s 31ms/step - loss: 1.8709 - accuracy: 0.4790 - val_loss: 0.9887 - val_accuracy: 0.6754
Epoch 2/100
15/15 [==============================] - 0s 18ms/step - loss: 0.9780 - accuracy: 0.6712 - val_loss: 0.6369 - val_accuracy: 0.7018
Epoch 3/100
15/15 [==============================] - 0s 19ms/step - loss: 0.6952 - accuracy: 0.6673 - val_loss: 0.6794 - val_accuracy: 0.6316
Epoch 4/100
15/15 [==============================] - 0s 18ms/step - loss: 0.6004 - accuracy: 0.7008 - val_loss: 0.6952 - val_accuracy: 0.6579
Epoch 5/100
15/15 [==============================] - 0s 19ms/step - loss: 0.6188 - accuracy: 0.6836 - val_loss: 0.7379 - val_accuracy: 0.6316
Epoch 6/100
15/15 [==============================] - 0s 19ms/step - loss: 0.6225 - accuracy: 0.6746 - val_loss: 0.7302 - val_accuracy: 0.6316
Epoch 7/100
15/15 [==============================] - 0s 20ms/step - loss: 0.6185 - accuracy: 0.6774 - val_loss: 0.7089 - val_accuracy: 0.6491
Epoch 

15/15 [==============================] - 0s 19ms/step - loss: 0.3561 - accuracy: 0.8486 - val_loss: 0.5186 - val_accuracy: 0.7895
Epoch 59/100
15/15 [==============================] - 0s 19ms/step - loss: 0.3533 - accuracy: 0.8486 - val_loss: 0.5137 - val_accuracy: 0.7895
Epoch 60/100
15/15 [==============================] - 0s 19ms/step - loss: 0.3535 - accuracy: 0.8486 - val_loss: 0.5145 - val_accuracy: 0.7895
Epoch 61/100
15/15 [==============================] - 0s 19ms/step - loss: 0.3493 - accuracy: 0.8486 - val_loss: 0.5099 - val_accuracy: 0.7895
Epoch 62/100
15/15 [==============================] - 0s 19ms/step - loss: 0.3476 - accuracy: 0.8536 - val_loss: 0.5089 - val_accuracy: 0.7982
Epoch 63/100
15/15 [==============================] - 0s 22ms/step - loss: 0.3464 - accuracy: 0.8542 - val_loss: 0.5092 - val_accuracy: 0.7895
Epoch 64/100
15/15 [==============================] - 0s 19ms/step - loss: 0.3441 - accuracy: 0.8542 - val_loss: 0.5044 - val_accuracy: 0.7982
Epoch 65/100